In [1]:
from Libs.load_data import DataLoader, get_dataset_split
from Libs.config import models_data_folder

import os
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, auc, roc_curve

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.initializers import Constant

2023-03-20 16:39:29.264936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 16:39:29.476431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 16:39:29.476476: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 16:39:30.370168: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data_loader = DataLoader(run=100, N=1000, s=0.5, t=0.01, d=0.2, m=1, override=False, folder=models_data_folder)
params = data_loader.get_params()
params

{'run': 100,
 'sigma': [0.5],
 'theta': [0.01],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}

In [3]:
Xs, best_labels = data_loader.get_grid()

In [4]:
dataset_split_params = {
    'window_size': 20, # how large is the window
    'overlap_size': None, # how many time interval of overlap there is between the windows
    'label_treshold': 1, # how many labels have to be at 1 in the window_size to consider the current window as a flare
    'split_on_run': True, # if True the windows of a run cannot be on different sets
    'shuffle_run': False, # if True shuffles the order of the runs before computing the windows
    'shuffle_window': False, # if True shuffles the order of the windows in the resulting dataframes
    'test_size': 0.3, # size of the test set expressed in percentage
    'val_size': 0.2, # size of the validation set expressed in percentage, considered only if get_validation is True
    'get_validation': True, # if True the output would be train,val,test set, otherwise it would be train,test
    'random_state': 42 # sets the seed for reproducibility
}

In [5]:
def make_model(input_shape, num_classes, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    if num_classes==2:
        output_layer = keras.layers.Dense(1, activation="sigmoid", bias_initializer=output_bias)(gap)
    else:
        output_layer = keras.layers.Dense(num_classes, activation="softmax", bias_initializer=output_bias)(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

def make_lstm(input_shape, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    lstm_model = Sequential()
    lstm_model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=input_shape))
    lstm_model.add(Dense(30, activation='relu'))
    lstm_model.add(Dense(10, activation='relu'))
    lstm_model.add(Dense(1, activation='sigmoid',bias_initializer=output_bias))
    return lstm_model


In [6]:
model_lbls = ['dt', 'nb', 'lp', 'svc', 'knn', 'cnn', 'lstm']

results = {}

models_folder = "models"
for overlap_size in np.sort([int(name.split('_')[1]) for name in os.listdir(models_folder)]):
    dataset_split_params['overlap_size'] = overlap_size
    df_train, df_val, df_test = get_dataset_split(Xs, best_labels, **dataset_split_params)

    X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
    X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
    X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_std = scaler.transform(X_train)
    X_val_std = scaler.transform(X_val)
    X_test_std = scaler.transform(X_test)
    # Make the data uniform to multivariate timeseries
    X_train_cnn = X_train_std.reshape((X_train_std.shape[0], X_train_std.shape[1], 1))
    X_val_cnn = X_val_std.reshape((X_val_std.shape[0], X_val_std.shape[1], 1))
    X_test_cnn = X_test_std.reshape((X_test_std.shape[0], X_test_std.shape[1], 1))
    
    results[overlap_size] = {}
    
    overlap_folder = os.path.join(models_folder, f'overlap_{overlap_size}')
    for m in os.listdir(overlap_folder):
        filename, extension = m.split('.')
        if extension != 'pickle':
            if extension == 'pkl':
                model_type = filename.split('_')[-1]
                clf = joblib.load(os.path.join(overlap_folder, m))
                y_pred = clf.predict(X_test)
            if extension == 'h5':
                model_type = filename.split('_')[0].lower()
                if  model_type == 'cnn':
                    model = make_model(input_shape=X_train_cnn.shape[1:], num_classes=2)
                if model_type == 'lstm':
                    model = make_lstm((X_train_std.shape[1], 1))
                model.load_weights(os.path.join(overlap_folder, m))
                if model_type == 'cnn':
                    y_pred = np.round(model.predict(X_test_cnn), 0)
                if model_type == 'lstm':
                    y_pred = np.round(model.predict(X_test_std), 0)
            fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=1)
            results[overlap_size][model_type] = {}
            results[overlap_size][model_type]['f1_macro'] = f1_score(y_test, y_pred, average='macro')
            results[overlap_size][model_type]['auc'] = auc(fpr, tpr)
                
results

2023-03-20 16:39:44.648942: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-20 16:39:44.649008: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-20 16:39:44.649049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5166cf34c918): /proc/driver/nvidia/version does not exist
2023-03-20 16:39:44.649288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


919/919 [==============================] - 5s 5ms/step


{0: {'cnn': {'f1_macro': 0.8619983946752034, 'auc': 0.8997071969540206},
  'dt': {'f1_macro': 0.8865640709057236, 'auc': 0.863065430617059},
  'knn': {'f1_macro': 0.8673563245156484, 'auc': 0.8272824152630055},
  'lp': {'f1_macro': 0.8117277501027982, 'auc': 0.7427730414269751},
  'nb': {'f1_macro': 0.8597675373531555, 'auc': 0.8368269668501428},
  'svc': {'f1_macro': 0.8600487634970393, 'auc': 0.9019989239746721},
  'lstm': {'f1_macro': 0.8791733836755473, 'auc': 0.8699406116790134}},
 5: {'cnn': {'f1_macro': 0.9039006833974363, 'auc': 0.8975346107181018},
  'dt': {'f1_macro': 0.913342576791809, 'auc': 0.9040368794901271},
  'knn': {'f1_macro': 0.9173728813559321, 'auc': 0.8854726198163554},
  'lp': {'f1_macro': 0.8776512238882481, 'auc': 0.9101091385036819},
  'nb': {'f1_macro': 0.8807420871628138, 'auc': 0.887609638009967},
  'svc': {'f1_macro': 0.8800020742943078, 'auc': 0.8980181907158433},
  'lstm': {'f1_macro': 0.901032047080942, 'auc': 0.8947113143241029}},
 10: {'cnn': {'f1_ma

# Bibliography
\[1\] _On the distribution of fluxes of gamma-ray blazars: hints for a stochastic process?_, Tavecchio et al., [https://arxiv.org/pdf/2004.09149.pdf](https://arxiv.org/pdf/2004.09149.pdf)
<!-- cite with: [\[1\]](https://arxiv.org/pdf/2004.09149.pdf)  -->
\[2\] _Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline_, Wang et al., [https://arxiv.org/abs/1611.06455](https://arxiv.org/abs/1611.06455)
<!-- cite with: [\[2\]](https://arxiv.org/abs/1611.06455)  -->
\[3\] _Solar Flare Prediction Based on the Fusion of Multiple Deep-learning Models_, Tang et al., [https://iopscience.iop.org/article/10.3847/1538-4365/ac249e/meta](https://iopscience.iop.org/article/10.3847/1538-4365/ac249e/meta)
<!-- cite with: [\[3\]](https://iopscience.iop.org/article/10.3847/1538-4365/ac249e/meta)  -->
\[4\] _Predicting Solar Energetic Particles Using SDO/HMI Vector Magnetic Data Products and a Bidirectional LSTM Network_, Abduallah et al., [https://iopscience.iop.org/article/10.3847/1538-4365/ac5f56/meta](https://iopscience.iop.org/article/10.3847/1538-4365/ac5f56/meta)
<!-- cite with: [\[4\]](https://iopscience.iop.org/article/10.3847/1538-4365/ac5f56/meta) -->